In [2]:
import os
os.chdir("E:/Ineuron/Project/Deep Learning Project/DeepCNNClassifier/DEEPCNNCLASSIFIER")

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallBacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_file_path: Path

In [4]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallBacksConfig:
        config = self.config.prepare_callbacks
        checkpoint_dir=os.path.dirname(config.checkpoint_model_file_path)
        
        create_directories([config.tensorboard_root_log_dir,checkpoint_dir])

        prepare_base_model_config = PrepareCallBacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_file_path=Path(config.checkpoint_model_file_path)
        )

        return prepare_base_model_config

In [5]:
import time

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

class PrepareCallback:
    def __init__(self, config: PrepareCallBacksConfig):
        self.config = config
    @property
    def _create_tb_ckpt_callbacks(self):
        timestamp=time.strftime("%Y-%m-%dT%H:%M:%S")
        tb_running_log_dir=os.path.join(self.config.tensorboard_root_log_dir,
        f"tb_logs_at {timestamp}")
        return [
            tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)]
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath=self.config.checkpoint_model_file_path,
        save_best_only=True)
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_ckpt_callbacks,
            self._create_ckpt_callbacks,
        ]

In [7]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callback = PrepareCallback(config=prepare_callback_config)
    call_back_list=prepare_callback.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2022-09-25 08:01:33,383: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2022-09-25 08:01:33,850: INFO: common]: yaml file: params.yaml loaded successfully
[2022-09-25 08:01:34,015: INFO: common]: created directory at: artifacts
[2022-09-25 08:01:34,017: INFO: common]: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir
[2022-09-25 08:01:34,019: INFO: common]: created directory at: artifacts/prepare_callbacks/checkpoint_dir
